# Football Match Prediction - Prediction Upload

This notebook predicts the next matchday using the trained classifier and encoder, then uploads the predictions to the betting platform.


In [1]:
import pickle
import pandas as pd
import numpy as np

# Load datasets and models
training_df = pickle.load(open("data/prepped_match_df.pck", "rb"))
next_matchday_df = pickle.load(open("data/next_matchday_df.pck", "rb"))
classifier = pickle.load(open("classifier_20250907_180042.pck", "rb"))
encoder = pickle.load(open("encoder.pck", "rb"))

print(f"Data loaded: {len(next_matchday_df)} matches to predict")


Data loaded: 9 matches to predict


In [2]:
# Prepare features for prediction
next_matchday_df_prep = next_matchday_df.drop(["date", "teamHomeName", "teamAwayName"], axis=1)
prediction_features = next_matchday_df_prep.drop(['resultClass'], axis=1)

# Align columns to match training data
training_features = training_df.drop(['resultClass'], axis=1)
prediction_features = prediction_features.reindex(columns=training_features.columns, fill_value=0)

# Make predictions
predictions_encoded = classifier.predict(prediction_features)
prediction_probabilities = classifier.predict_proba(prediction_features)
predictions_decoded = encoder.inverse_transform(predictions_encoded)

# Get probability for each predicted class
prediction_probabilities_max = []
for i, pred_encoded in enumerate(predictions_encoded):
    class_index = np.where(classifier.classes_ == pred_encoded)[0][0]
    prob = prediction_probabilities[i, class_index]
    prediction_probabilities_max.append(prob)

# Create results dataframe
results_df = pd.DataFrame({
    'Date': next_matchday_df['date'],
    'Home_Team': next_matchday_df['teamHomeName'],
    'Away_Team': next_matchday_df['teamAwayName'],
    'Matchday': next_matchday_df['matchDay'],
    'Season': next_matchday_df['season'],
    'Prediction': predictions_decoded,
    'Prediction_Probability': prediction_probabilities_max,
    'Home_Value': next_matchday_df['teamHomeValue'],
    'Away_Value': next_matchday_df['teamAwayValue']
})

results_df


,Date,Home_Team,Away_Team,Matchday,Season,Prediction,Prediction_Probability,Home_Value,Away_Value
9828,2025-09-26 20:30:00,FC Bayern München,SV Werder Bremen,5,2025,2:1,0.125147,857.50,143.00
9829,2025-09-27 15:30:00,1. FSV Mainz 05,Borussia Dortmund,5,2025,1:1,0.109582,138.58,438.10
9830,2025-09-27 15:30:00,VfL Wolfsburg,RB Leipzig,5,2025,1:1,0.137715,264.40,422.75
9831,2025-09-27 15:30:00,FC St. Pauli,Bayer 04 Leverkusen,5,2025,1:1,0.121752,52.15,439.75
9832,2025-09-27 15:30:00,1. FC Heidenheim 1846,FC Augsburg,5,2025,1:1,0.111212,58.90,127.83
9833,2025-09-27 18:30:00,Borussia Mönchengladbach,Eintracht Frankfurt,5,2025,1:1,0.104100,153.85,336.40
9834,2025-09-28 15:30:00,SC Freiburg,TSG Hoffenheim,5,2025,1:1,0.176335,167.70,154.85
9835,2025-09-28 17:30:00,1. FC Köln,VfB Stuttgart,5,2025,1:1,0.111521,74.68,277.63
9836,2025-09-28 19:30:00,1. FC Union Berlin,Hamburger SV,5,2025,1:1,0.138596,124.40,80.35


In [3]:
# Summary
print("=== MATCH PREDICTIONS ===")
for idx, row in results_df.iterrows():
    confidence = row['Prediction_Probability'] * 100
    print(f"{row['Home_Team']} vs {row['Away_Team']}")
    print(f"  → {row['Prediction']} (Confidence: {confidence:.1f}%)")
    print()


=== MATCH PREDICTIONS ===
FC Bayern München vs SV Werder Bremen
  → 2:1 (Confidence: 12.5%)

1. FSV Mainz 05 vs Borussia Dortmund
  → 1:1 (Confidence: 11.0%)

VfL Wolfsburg vs RB Leipzig
  → 1:1 (Confidence: 13.8%)

FC St. Pauli vs Bayer 04 Leverkusen
  → 1:1 (Confidence: 12.2%)

1. FC Heidenheim 1846 vs FC Augsburg
  → 1:1 (Confidence: 11.1%)

Borussia Mönchengladbach vs Eintracht Frankfurt
  → 1:1 (Confidence: 10.4%)

SC Freiburg vs TSG Hoffenheim
  → 1:1 (Confidence: 17.6%)

1. FC Köln vs VfB Stuttgart
  → 1:1 (Confidence: 11.2%)

1. FC Union Berlin vs Hamburger SV
  → 1:1 (Confidence: 13.9%)



In [8]:
# Upload predictions to betting platform
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from difflib import get_close_matches
import dotenv
import os

# Load environment variables
dotenv.load_dotenv(".env")

# Start browser and login
driver = webdriver.Chrome()
driver.get(os.getenv("LINK-TIPPABGABE"))

# Login process
nav_button = driver.find_element(By.CLASS_NAME, "navtoggle")
nav_button.click()

email_input = driver.find_element(By.ID, "kennung")
email_input.send_keys(os.getenv("EMAIL"))
passwort_input = driver.find_element(By.ID, "passwort")
passwort_input.send_keys(os.getenv("PASSWORT"))

login_button = driver.find_element(By.NAME, "submitbutton")
login_button.click()


In [9]:
# Handle cookie banner if present
try:
    frame = driver.switch_to.frame(driver.find_element(By.XPATH, "/html/body/div[2]/iframe"))
    driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div[3]/div[1]/div[2]/button").click()
    driver.switch_to.default_content()
except:
    print("No cookie banner found or already handled")
    driver.switch_to.default_content()


In [10]:
# Fill in predictions
tippabgabe_tabelle = driver.find_element(By.ID, "tippabgabeSpiele")
for datarow in tippabgabe_tabelle.find_elements(By.CLASS_NAME, "datarow"):
    homeTeam = datarow.find_element(By.CLASS_NAME, "col1").get_attribute("innerHTML")
    if homeTeam != "":
        # Find matching team in our predictions
        selectedTeam = get_close_matches(homeTeam, results_df["Home_Team"])[0]
        prediction = results_df[results_df["Home_Team"] == selectedTeam]["Prediction"].iloc[0].split(":")
        
        # Fill in the prediction
        inputs = datarow.find_elements(By.TAG_NAME, "input")
        inputs[1].clear()
        inputs[1].send_keys(prediction[0])
        inputs[2].clear()
        inputs[2].send_keys(prediction[1])
        
        print(f"Filled: {selectedTeam} -> {prediction[0]}:{prediction[1]}")

print("All predictions filled!")


Filled: FC Bayern München -> 2:1
Filled: 1. FC Heidenheim 1846 -> 1:1
Filled: FC St. Pauli -> 1:1
Filled: 1. FSV Mainz 05 -> 1:1
Filled: VfL Wolfsburg -> 1:1
Filled: Borussia Mönchengladbach -> 1:1
Filled: SC Freiburg -> 1:1
Filled: 1. FC Köln -> 1:1
Filled: 1. FC Union Berlin -> 1:1
All predictions filled!


In [11]:
# Submit predictions (optional - uncomment to actually submit)
driver.find_element(By.NAME, "submitbutton").click()
print("Predictions submitted!")

Predictions submitted!


In [12]:
# Close browser
driver.close()
print("Browser closed.")


Browser closed.
